# Import modules

In [1]:
%%time
%load_ext memory_profiler

import datetime
import os, glob, sys, gc
import warnings
# warnings.filterwarnings('ignore', '.*invalid value encountered in true_divide.*', )

from dask.diagnostics import ProgressBar
pbar = ProgressBar(minimum=10)
pbar.register()
#pbar.unregister()

import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap

import cartopy.crs       as ccrs
import cartopy.feature as cfeature
import cartopy

import cmcrameri

mpl.rcParams['savefig.dpi'] = 300

CPU times: user 1.03 s, sys: 1.32 s, total: 2.35 s
Wall time: 1.19 s


# Starters

In [2]:
%%time
%%memit -c
print(datetime.datetime.now())

from IPython.core.magic import register_cell_magic
import traceback

dirout = '24-08-22-compute-aou-noresm/'
if not os.path.isdir(dirout) : os.mkdir(dirout)

netcdfdir = dirout+'netcdf_files/'
if not os.path.isdir(netcdfdir) : os.mkdir(netcdfdir)

sys.stdout.echo = open(dirout+'stdout.txt', 'w')
sys.stderr.echo = open(dirout+'stderr.txt', 'w')

2024-08-22 16:35:30.019427
peak memory: 316.14 MiB, increment: 124.60 MiB
CPU times: user 51.3 ms, sys: 20.9 ms, total: 72.2 ms
Wall time: 158 ms


In [29]:
# #-------------------
# # Save errors and outputs in files
# #-------------------
# 
# sys.stdout.echo = open(dirout+'stdout.txt', 'w')
# # sys.stderr.echo = open(dirout+'stderr.txt', 'w')
# 
# # Create error.log and erase former one
# error_file = dirout+'error.log'
# fff = open(error_file, 'w')
# fff.close()
# 
# @register_cell_magic
# def capture_errors(line, cell):
#     # Get the file path from the magic line arguments
#     # file_path = dirout+line.strip()
#     file_path = error_file
#     ip = get_ipython()
#     try:
#         # Split the cell content by lines and detect magic commands
#         cell_lines = cell.strip().split('\n')
#         magic_lines = [line for line in cell_lines if line.startswith('%%')]
#         
#         # Execute each magic command in the cell
#         ilines = 0
#         for magic_line in magic_lines:
#             magic_command = magic_line.split()[0][2:]  # Extract the magic command
#             magic_param = ' '.join(magic_line.split()[1:])
#             ip.run_line_magic(magic_command, magic_param)
#             ilines += 1
#         #
#         cell_code = "\n".join(cell_lines[ilines:])  # Get the code below the magic command line     
#         # Execute the cell code
#         exec(cell_code, globals(), locals())
#     except Exception as e:
#         error_message = traceback.format_exc()
#         # Capture any exceptions and write the traceback to the file in append mode
#         with open(file_path, "a") as f:
#             f.write(error_message)
#             f.write("\n")  # Add a new line after each error entry
#         # Print the error message to the notebook
#         print(f"An error occurred:\n{error_message}")
#         print(f"The error was also written to {file_path}")
#

2024-08-22 16:32:28.840452
peak memory: 385.30 MiB, increment: 167.43 MiB
CPU times: user 94 ms, sys: 13.8 ms, total: 108 ms
Wall time: 215 ms


# Main parameters

In [3]:
%%time
%%memit -c
print(datetime.datetime.now())

kwopends = dict(use_cftime=True, decode_times=None,
                decode_cf=True, decode_coords=True)

kwopenmfds = dict(combine='by_coords', parallel=True, 
                  use_cftime=True, decode_times=None,
                  decode_cf=True, decode_coords=True)

rename_dict = {
    "x": "i",
    "y": "j",
    "lat": "latitude", 
    "lon": "longitude",
    "nav_lat": "latitude", 
    "nav_lon": "longitude",
    'lev': 'depth', 
    'deptht': 'depth', 
    'olevel': 'depth', 
    "Depth":"depth"
}

print(kwopends)

2024-08-22 16:35:39.852423
{'use_cftime': True, 'decode_times': None, 'decode_cf': True, 'decode_coords': True}
peak memory: 316.59 MiB, increment: 133.04 MiB
CPU times: user 38.9 ms, sys: 12.8 ms, total: 51.7 ms
Wall time: 156 ms


# Define some functions

## Preparation of data 

In [4]:
def shift_180_lon(zwda, verbose=False): 
    if verbose: print("func: shift_180_lon")
    
    try: 
        if not np.nanmin(zwda['longitude']) < -150: 
            zwda['longitude'] = (zwda['longitude'] + 180) % 360 - 180
            addtxt=str(datetime.datetime.now())+' shift_180_lon to get longitude from -180 to 180'
            try: zwda.attrs['history'] =  addtxt + ' ; '+zwda.attrs['history']
            except: zwda.attrs['history'] =  addtxt             
        #
    except: print('WARNING! longitude likely not shifted')
    return zwda
#

def rename_vars_dims_coords(ds, rename_dict, verbose=False):
    """
    Renames variables, dimensions, and coordinates in an xarray Dataset according to the provided rename dictionary.

    Parameters:
    -----------
    ds : xr.Dataset
        The xarray Dataset to be renamed.
    rename_dict : Dict[str, str]
        Dictionary containing the variable, dimension, or coordinate names to be renamed. 
        The keys represent the original names, and the values represent the new names.
    verbose : bool, optional
        If True, prints the function name at the start and end of execution (default is False).
    
    Returns:
    --------
    xr.Dataset
        A new xarray Dataset with variables, dimensions, and coordinates renamed according to the rename dictionary.
    
    Example:
    --------
    import xarray as xr
    data = {'temp': ([], [0]), 'sali': ([], [1])}
    coords = {'time': [0]}
    ds = xr.Dataset(data, coords)
    renamed_ds = rename_vars_dims_coords(ds, {'temp': 'temperature', 'sali': 'salinity'})

    Dependencies:
    -------------
    xarray
    """
    if verbose: print('func: rename_vars_dims_coords')
    for old_name, new_name in rename_dict.items():
        if (old_name in ds.variables) | (old_name in ds.dims) | (old_name in ds.coords): 
            ds = ds.rename({old_name: new_name})
        #
    if verbose: print('endfunc')
    return ds
#

def split_coords_dimensions(ds, verbose=False):
    """
    Splits the latitude, longitude, and depth dimensions and coordinates of an xarray dataset into separate variables,
    updates their names, and assigns them back to the dataset.

    Parameters:
    -----------
    ds : xr.Dataset
        The xarray Dataset to be updated.
    verbose : bool, optional
        If True, prints the function name at the start and end of execution (default is False).
    
    Returns:
    --------
    xr.Dataset
        A new xarray Dataset with the latitude, longitude, and depth dimensions and coordinates split into separate variables
        and reassigned to the original dataset.
    
    Example:
    --------
    import xarray as xr
    data = {'temp': ([0, 1, 2], [0, 1]), 'sali': ([0, 1, 2], [0, 1])}
    coords = {'latitude': [0, 1, 2], 'longitude': [0, 1], 'depth': [0, 1, 2]}
    ds = xr.Dataset(data, coords)
    updated_ds = split_coords_dimensions(ds)

    Dependencies:
    -------------
    xarray
    """
    if verbose: print('func: split_coords_dimensions')
    new_coords = {}
    new_coords2 = {}
    new_dims = {}
    dim_name_dict = dict(latitude='j', longitude='i', depth='k')
    dimschanged = []
    for name, coord in ds.coords.items():
        if name in ds.dims and name in ["latitude", "longitude", "depth"]:
            new_coords[name + "_coord"] = coord
            new_dims[name] = dim_name_dict[name]
            new_coords2[name + "_coord"] = name
            dimschanged.append(name)
    if verbose: print('endfunc')
    for name in ['k', 'j', 'i']: 
        if name in ds.coords: dimschanged.append(name)
    #
    return ds.assign_coords(new_coords).rename_dims(new_dims).drop_vars(dimschanged).rename(new_coords2)
#


## Others

In [5]:
def get_esgf_dataset_filepaths(variable, sourceID, experimentID, 
                               freq='mon', grid='g*', version='latest', 
                               variant='r1i1p1f1',
                               mipera = 'CMIP6', diresgf='/mnt/reef-ns1002k-esgf/', verbose=False, **kwargs): 
    """
    Returns the filepaths of the remote netCDF files corresponding to the specified dataset of the Earth System
    Grid Federation (ESGF) data portal on NIRD.

    Parameters:
    -----------
    variable : str
        Variable to search for on ESGF data portal.
    sourceID : str
        Name of the data source on the ESGF data portal.
    experimentID : str
        Name of the experiment on the ESGF data portal.
    freq : str, optional
        Frequency of the data (default is 'mon').
    grid : str, optional
        Type of grid (default is 'g*').
    version : str, optional
        Version of the data being queried (default is 'latest').
    variant : str, optional
        Label for the variant of the data being queried (default is 'r1i1p1f1').
    mipera : str, optional
        Name of the CMIP era being queried (default is 'CMIP6').
    diresgf : str, optional
        Absolute path to the directory where the data is stored (default is '/mnt/reef-ns1002k-esgf/').
    verbose : bool, optional
        If True, prints the function name at the start and end of execution (default is False).
    **kwargs : dict, optional
        Other key-value arguments to be passed in the function.

    Returns:
    --------
    List[str]
        A list of filepaths corresponding to the specified dataset on the ESGF data portal.

    Example:
    --------
    fp_list = get_esgf_dataset_filepaths('tas', 'CanESM5', 'historical', freq='mon')

    Dependencies:
    -------------
    glob, sys
    """
    import glob, sys
    
    if verbose: print('func: get_esgf_dataset_filepaths')
    
    if experimentID in ['1pctCO2', 'piControl', 'historical', 'abrupt-4xCO2']: zwActivity='CMIP'
    elif experimentID in ['ssp126', 'ssp245', 'ssp585']: zwActivity='ScenarioMIP'
    else: sys.exit('Check experimentID, case not implemented')
    
    if sourceID in ['CESM2', 'CESM2-WACCM']: zwInstitutionID = 'NCAR'
    elif sourceID in ['ACCESS-ESM1-5']: zwInstitutionID = 'CSIRO'
    elif sourceID in ['CNRM-ESM2-1']: zwInstitutionID = 'CNRM-CERFACS'
    elif sourceID in ['CanESM5', 'CanESM5-CanOE']: zwInstitutionID = 'CCCma'
    elif sourceID in ['UKESM1-0-LL']: zwInstitutionID = 'NIMS-KMA'
    elif sourceID in ['GFDL-CM4', 'GFDL-ESM4']: zwInstitutionID = 'NOAA-GFDL'
    elif sourceID in ['IPSL-CM6A-LR', 'IPSL-CM6A-LR-INCA']: zwInstitutionID = 'IPSL'
    elif sourceID in ['MIROC-ES2L']: zwInstitutionID = 'MIROC'
    elif sourceID in ['MPI-ESM1-2-LR', 'ICON-ESM-LR']: zwInstitutionID = 'MPI-M'
    elif sourceID in ['NorESM2-LM']: zwInstitutionID = 'NCC'
    else: sys.exit('Check sourceID, case not implemented')
    
    ocean_list = ['fgco2', 'intpp', 'o2', 'thetao', 'so', 'agessc', 'po4', 'no3']
    if variable in ocean_list: zwTableID = 'O'+freq
    elif variable in ['areacello']: zwTableID='Ofx'
    elif variable in ['psl']: zwTableID='A'+freq
    else: sys.exit('!!! WARNING !!! Check variable, case not implemented')
        
    zwdname = diresgf + mipera +'/'+ zwActivity +'/'+ \
        zwInstitutionID +'/'+ sourceID +'/'+ \
        experimentID  +'/'+ variant +'/'+ zwTableID +'/'+ \
        variable+'/'+ grid +'/'+ version +'/'
    zwfname = variable +'_'+ zwTableID +'_'+ sourceID +'_'+ \
        experimentID +'_'+ variant +'_'+ grid +'*.nc' 

    if verbose: print('endfunc')
    return glob.glob(zwdname + zwfname)
#
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
        nb: y[~nans] values of y that are not nans
            x(~nans) indexes of y that are not nans
    """

    return np.isnan(y), lambda z: z.nonzero()[0]
#


## O2SAT and AOU

In [6]:
def get_o2sat_garcia(temp, saln, verbose=False):
    """
    Calculates the dissolved oxygen concentration at saturation using the Garcia and Gordon (1992) equation.

    Parameters:
    -----------
    temp : xr.DataArray
        DataArray containing the temperature values in degrees Celsius.
    saln : xr.DataArray
        DataArray containing the salinity values in practical salinity units.
    verbose : bool
        Print verbose output if True.

    Returns:
    --------
    xr.DataArray
        DataArray containing the dissolved oxygen concentration at saturation in mol/m3.

    Example:
    --------
    o2sat_data = get_o2sat_garcia(temp_data, sal_data)

    Dependencies:
    -------------
    numpy
    """
    
    import numpy as np
    
    if verbose: print('func: get_o2sat_garcia')
    # Garcia, H. E., & Gordon, L. I. (1993). Erratum: Oxygen solubility in seawater: better fitting equations. Limnology and Oceanography, 38, 656.
    # Garcia, H. E., & Gordon, L. I. (1992). Oxygen solubility in seawater: Better fitting equations. Limnology and Oceanography, 37(6), 1307–1312. https://doi.org/10.4319/lo.1992.37.6.1307

    #micromol/kg
    A0=5.80818
    A1=3.20684
    A2=4.11890
    A3=4.93845
    A4=1.01567
    A5=1.41575
    B0=-7.01211e-3
    B1=-7.25958e-3
    B2=-7.93334e-3
    B3=-5.54491e-3
    C0=-1.32412e-7
    
    Ts = np.log((298.15-temp)/(273.15+temp))
    OXY_nor = A0 + A1*Ts + A2*Ts**2 + A3*Ts**2 + A3*Ts**3 + A4*Ts**4 + A5*Ts**5 + saln*(B0 + B1*Ts + B2*Ts**2 + B3*Ts**3) + C0*saln**2
    o2sat = np.exp(OXY_nor)*1.024e-3 # micromol/kg=> mol/m3
        
    o2sat.attrs['units']='mol m-3'
    o2sat.attrs['longname']='Dissolved Oxygen Concentration at Saturation'
    o2sat.attrs['description']='Dissolved Oxygen Concentration at Saturation computed following Garcia, H. E., & Gordon, L. I. (1993). \
    Erratum: Oxygen solubility in seawater: better fitting equations. Limnology and Oceanography, 38, 656. See also Garcia, H. E., & \
    Gordon, L. I. (1992). Oxygen solubility in seawater: Better fitting equations. Limnology and Oceanography, 37(6), 1307–1312.\
    Computed from temperature and salinity.'
    o2sat = o2sat.rename('o2sat')
    if verbose: print('endfunc')
    
    return o2sat #mol O2/m3
#

# Compute AOU for SSP585

## Check data avaibility

In [8]:
%%time
%%memit -c
print(datetime.datetime.now())
print('# Compute AOU for SSP585')
print('## Check data avaibility')

simu = 'ssp585'
esm  = 'NorESM2-LM'
diresgf = '/mnt/reef-ns1002k-ns9034k/'
version = 'v20191108'
variant = 'r1i1p1f1'
grid = 'gr'

var_list = ['thetao', 'so', 'o2']
for var in var_list: 
    fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                            diresgf=diresgf, version=version, 
                                            variant=variant, grid=grid)
    zwds = xr.open_mfdataset(fname_list, **kwopenmfds)
    ymin = int(np.min(zwds['time.year']))
    ymax = int(np.max(zwds['time.year']))
    tstep = zwds[var].shape[0]
    good = tstep/12 == ymax-ymin+1
    if good: 
        print(f'{var.upper()}, GOOD')
        print(f'{ymin} to {ymax}, {tstep/12} years ({tstep} months)')
        print('------------')
    else:
        print('Years missing')
        for fname in fname_list: print(fname)
    #
#  





2024-08-22 10:56:26.959818
# Compute AOU for SSP585
THETAO, GOOD
2015 to 2100, 86.0 years (1032 months)
------------
SO, GOOD
2015 to 2100, 86.0 years (1032 months)
------------
O2, GOOD
2015 to 2100, 86.0 years (1032 months)
------------
peak memory: 619.36 MiB, increment: 410.98 MiB
CPU times: user 1.34 s, sys: 961 ms, total: 2.3 s
Wall time: 9.21 s


## Compute

In [19]:
%%time
%%memit -c
print(datetime.datetime.now())
print('# Compute AOU for SSP585')
print('## Compute')


simu = 'ssp585'
esm  = 'NorESM2-LM'
diresgf = '/mnt/reef-ns1002k-ns9034k/'
version = 'v20191108'
variant = 'r1i1p1f1'
grid = 'gr'

year_list = ['%04d' %yyy for yyy in np.arange(2015, 2099.5)]

# Load temperature
var = 'thetao'
fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                        diresgf=diresgf, version=version, 
                                        variant=variant, grid=grid)
zwds = xr.open_mfdataset(fname_list, **kwopenmfds)
zwds2 = zwds[var].to_dataset()
zwds2 = rename_vars_dims_coords(zwds2, rename_dict)
zwds2 = split_coords_dimensions(zwds2)
zwds_temp = shift_180_lon(zwds2)

# Load salinity
var = 'so'
fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                        diresgf=diresgf, version=version, 
                                        variant=variant, grid=grid)
zwds = xr.open_mfdataset(fname_list, **kwopenmfds)
zwds2 = zwds[var].to_dataset()
zwds2 = rename_vars_dims_coords(zwds2, rename_dict)
zwds2 = split_coords_dimensions(zwds2)
zwds_sali = shift_180_lon(zwds2)

# Load oxygen
var = 'o2'
fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                        diresgf=diresgf, version=version, 
                                        variant=variant, grid=grid)
zwds = xr.open_mfdataset(fname_list, **kwopenmfds)
zwds2 = zwds[var].to_dataset()
zwds2 = rename_vars_dims_coords(zwds2, rename_dict)
zwds2 = split_coords_dimensions(zwds2)
zwds_oxyg = shift_180_lon(zwds2)

# save for later
zwds_attrs=zwds.attrs

del zwds, zwds2
gc.collect()

# Loop on years
for year in year_list: 

    temp = zwds_temp.sel(time=year)['thetao'].load()
    sali = zwds_sali.sel(time=year)['so'].load()
    oxyg = zwds_oxyg.sel(time=year)['o2'].load()

    # Compute o2sat
    o2sat = get_o2sat_garcia(temp, sali, verbose=False).compute()

    # Clean
    del sali, temp
    gc.collect()

    # Compute aou
    aou = xr.zeros_like(o2sat)
    aou.values = o2sat.values - oxyg.values
    aou.attrs={}
    aou.attrs['units']='mol m-3'
    aou.attrs['long_name']='Apparent Oxygen Utilization'
    aou.attrs['description']='AOU computed as O2sat - O2 with O2sat computed following Garcia, H. E., & \
    Gordon, L. I. (1992). Oxygen solubility in seawater: Better fitting equations. Limnology and Oceanography, 37(6), 1307–1312.'
    aou = aou.rename('aou')

    # Clean
    del oxyg
    gc.collect()

    # Temporal mean
    o2sat_tavg = o2sat.groupby('time.year').mean(dim='time')
    aou_tavg = aou.groupby('time.year').mean(dim='time')

    # Clean
    del o2sat, aou
    gc.collect()

    # create dataset
    o2sat_ds = o2sat_tavg.to_dataset() 
    o2sat_ds.attrs = zwds_attrs
    aou_ds = aou_tavg.to_dataset() 
    aou_ds.attrs = zwds_attrs

    # Save in netcdf
    #---------------    
    # print('Save in netcdf...')
    ncname = netcdfdir+esm+'_'+simu+'_o2sat_'+year+'.nc'
    o2sat_ds.to_netcdf(ncname)
    # print('File saved: %s'%ncname)
    #---------------    
    # print('Save in netcdf...')
    ncname = netcdfdir+esm+'_'+simu+'_aou_'+year+'.nc'
    aou_ds.to_netcdf(ncname)
    # print('File saved: %s'%ncname)
#
print('Done')






2024-08-22 11:48:32.109096
# Compute AOU for SSP585
## Compute
Done
peak memory: 5108.88 MiB, increment: 4546.36 MiB
CPU times: user 15min 32s, sys: 2min 33s, total: 18min 6s
Wall time: 18min 42s


# Compute AOU for historical

## Check data avaibility

In [9]:
%%time
%%memit -c
print(datetime.datetime.now())
print('# Compute AOU for hitsorical')
print('## Check data avaibility')

simu='historical'
esm  = 'NorESM2-LM'
diresgf = '/mnt/reef-ns1002k-ns9034k/'
version = 'v20190815'
variant = 'r1i1p1f1'
grid = 'gr'

var_list = ['thetao', 'so', 'o2']
for var in var_list: 
    fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                            diresgf=diresgf, version=version, 
                                            variant=variant, grid=grid)
    zwds = xr.open_mfdataset(fname_list, **kwopenmfds)
    ymin = int(np.min(zwds['time.year']))
    ymax = int(np.max(zwds['time.year']))
    tstep = zwds[var].shape[0]
    good = tstep/12 == ymax-ymin+1
    if good: 
        print(f'{var.upper()}, GOOD')
        print(f'{ymin} to {ymax}, {tstep/12} years ({tstep} months)')
        print('------------')
    else:
        print('Years missing')
        for fname in fname_list: print(fname)
    #
#  






2024-08-22 11:38:45.026671
# Compute AOU for hitsorical
## Check data avaibility
THETAO, GOOD
1850 to 2014, 165.0 years (1980 months)
------------
SO, GOOD
1850 to 2014, 165.0 years (1980 months)
------------
O2, GOOD
1850 to 2014, 165.0 years (1980 months)
------------
peak memory: 904.45 MiB, increment: 443.49 MiB
CPU times: user 1.52 s, sys: 453 ms, total: 1.98 s
Wall time: 13.8 s


## Compute

In [20]:
%%time
%%memit -c
print(datetime.datetime.now())
print('# Compute AOU for historical')
print('## Compute')

simu='historical'
esm  = 'NorESM2-LM'
diresgf = '/mnt/reef-ns1002k-ns9034k/'
version = 'v20190815'
variant = 'r1i1p1f1'
grid = 'gr'

year_list = ['%04d' %yyy for yyy in np.arange(1850, 2014.5)]

# Load temperature
var = 'thetao'
fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                        diresgf=diresgf, version=version, 
                                        variant=variant, grid=grid)
zwds = xr.open_mfdataset(fname_list, **kwopenmfds)
zwds2 = zwds[var].to_dataset()
zwds2 = rename_vars_dims_coords(zwds2, rename_dict)
zwds2 = split_coords_dimensions(zwds2)
zwds_temp = shift_180_lon(zwds2)

# Load salinity
var = 'so'
fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                        diresgf=diresgf, version=version, 
                                        variant=variant, grid=grid)
zwds = xr.open_mfdataset(fname_list, **kwopenmfds)
zwds2 = zwds[var].to_dataset()
zwds2 = rename_vars_dims_coords(zwds2, rename_dict)
zwds2 = split_coords_dimensions(zwds2)
zwds_sali = shift_180_lon(zwds2)

# Load oxygen
var = 'o2'
fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                        diresgf=diresgf, version=version, 
                                        variant=variant, grid=grid)
zwds = xr.open_mfdataset(fname_list, **kwopenmfds)
zwds2 = zwds[var].to_dataset()
zwds2 = rename_vars_dims_coords(zwds2, rename_dict)
zwds2 = split_coords_dimensions(zwds2)
zwds_oxyg = shift_180_lon(zwds2)

# save for later
zwds_attrs=zwds.attrs

del zwds, zwds2
gc.collect()

# Loop on years
for year in year_list: 

    temp = zwds_temp.sel(time=year)['thetao'].load()
    sali = zwds_sali.sel(time=year)['so'].load()
    oxyg = zwds_oxyg.sel(time=year)['o2'].load()

    # Compute o2sat
    o2sat = get_o2sat_garcia(temp, sali, verbose=False).compute()

    # Clean
    del sali, temp
    gc.collect()

    # Compute aou
    aou = xr.zeros_like(o2sat)
    aou.values = o2sat.values - oxyg.values
    aou.attrs={}
    aou.attrs['units']='mol m-3'
    aou.attrs['long_name']='Apparent Oxygen Utilization'
    aou.attrs['description']='AOU computed as O2sat - O2 with O2sat computed following Garcia, H. E., & \
    Gordon, L. I. (1992). Oxygen solubility in seawater: Better fitting equations. Limnology and Oceanography, 37(6), 1307–1312.'
    aou = aou.rename('aou')

    # Clean
    del oxyg
    gc.collect()

    # Temporal mean
    o2sat_tavg = o2sat.groupby('time.year').mean(dim='time')
    aou_tavg = aou.groupby('time.year').mean(dim='time')

    # Clean
    del o2sat, aou
    gc.collect()

    # create dataset
    o2sat_ds = o2sat_tavg.to_dataset() 
    o2sat_ds.attrs = zwds_attrs
    aou_ds = aou_tavg.to_dataset() 
    aou_ds.attrs = zwds_attrs

    # Save in netcdf
    #---------------    
    # print('Save in netcdf...')
    ncname = netcdfdir+esm+'_'+simu+'_o2sat_'+year+'.nc'
    o2sat_ds.to_netcdf(ncname)
    # print('File saved: %s'%ncname)
    #---------------    
    # print('Save in netcdf...')
    ncname = netcdfdir+esm+'_'+simu+'_aou_'+year+'.nc'
    aou_ds.to_netcdf(ncname)
    # print('File saved: %s'%ncname)
#
print('Done')





2024-08-22 12:07:14.437671
# Compute AOU for historical
## Compute
Done
peak memory: 6779.99 MiB, increment: 5238.19 MiB
CPU times: user 30min 28s, sys: 4min 52s, total: 35min 20s
Wall time: 36min 24s


# Compute AOU for piControl with specific years depending on models

## def shorten_fname_list(fname_list, startyear, endyear):

In [7]:
def shorten_fname_list(fname_list, startyear, endyear):
    fname_list.sort()
    new_fname_list = []
    for fname in fname_list: 
        year1_of_fname = int(fname.split('/')[-1].split('_')[-1].split('-')[0][:4])
        year2_of_fname = int(fname.split('/')[-1].split('_')[-1].split('-')[1][:4])
        startyear_in_between = ((startyear>=year1_of_fname) & (startyear<=year2_of_fname))
        endyear_in_between   = ((endyear>=year1_of_fname) & (endyear<=year2_of_fname))
        year1_in_between = ((year1_of_fname>=startyear) & (year1_of_fname<=endyear))
        year2_in_between = ((year2_of_fname>=startyear) & (year2_of_fname<=endyear))
        if startyear_in_between | endyear_in_between | year1_in_between | year2_in_between: 
            if not (fname in new_fname_list): 
                new_fname_list.append(fname)
        #
    #
    if len(new_fname_list)==0: new_fname_list=fname_list
    return new_fname_list
#


## Check data avaibility

In [8]:
%%time
%%memit -c
print(datetime.datetime.now())
print('# Compute AOU for piControl with specific years depending on models')
print('## Check data avaibility')

simu='piControl'
esm  = 'NorESM2-LM'
diresgf = '/mnt/reef-ns1002k-ns9034k/'
version = 'v20210118'
variant = 'r1i1p1f1'
grid = 'gr'

refyear_dict = {
    'MPI-ESM1-2-LR': 1850,
    'ACCESS-ESM1-5':  161,
    'IPSL-CM6A-LR' : 1910,
    'CanESM5'      : 5201,
    'MIROC-ES2L'   : 1850,
    'NorESM2-LM'   : 1600
}

import time

# Pause execution for 10 seconds
#time.sleep(10)

var_list = ['thetao', 'so', 'o2']
for var in var_list: 

    print('------------')
    startyear, endyear = refyear_dict[esm], refyear_dict[esm]+165+84
    print('piControl targeted time period: %04d-%04d'%(startyear, endyear))

    fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                            diresgf=diresgf, version=version, 
                                            variant=variant, grid=grid)
    new_fname_list = shorten_fname_list(fname_list, startyear, endyear)
    zwds = xr.open_mfdataset(new_fname_list, **kwopenmfds)
    ymin = int(np.min(zwds['time.year']))
    ymax = int(np.max(zwds['time.year']))
    tstep = zwds[var].shape[0]
    good1 = tstep/12 == ymax-ymin+1
    good2 = (startyear>=ymin) & (endyear<=ymax)
    if good1 & good2: 
        print(f'>>> {var.upper()}: GOOD, time period complete and match target')
        print(f'{ymin} to {ymax}, {tstep/12} years ({tstep} months)')
    elif good1 and (not good2): 
        print(f'!!! {var.upper()}: time period complete BUT do not match target')
        print(f'{ymin} to {ymax}, {tstep/12} years ({tstep} months), file list:')
        for fname in fname_list: print(fname)
    else: 
        print(f'!!! {var.upper()}: some years are missing. Here is the file list: ')
        for fname in fname_list: print(fname)
    #
#  
print('------------')
print('Done')



2024-08-22 16:36:10.777262
# Compute AOU for piControl with specific years depending on models
## Check data avaibility
------------
piControl targeted time period: 1600-1849
>>> THETAO: GOOD, time period complete and match target
1600 to 1850, 251.0 years (3012 months)
------------
piControl targeted time period: 1600-1849
>>> SO: GOOD, time period complete and match target
1600 to 1850, 251.0 years (3012 months)
------------
piControl targeted time period: 1600-1849
>>> O2: GOOD, time period complete and match target
1600 to 1850, 251.0 years (3012 months)
------------
Done
peak memory: 701.09 MiB, increment: 517.32 MiB
CPU times: user 2.75 s, sys: 1.42 s, total: 4.17 s
Wall time: 22.3 s


## Compute

In [10]:
%%time
%%memit -c

print(datetime.datetime.now())
print('# Compute AOU for piControl with specific years depending on models')
print('## Compute')


simu='piControl'
esm  = 'NorESM2-LM'
diresgf = '/mnt/reef-ns1002k-ns9034k/'
version = 'v20210118'
variant = 'r1i1p1f1'
grid = 'gr'

refyear_dict = {
    'MPI-ESM1-2-LR': 1850,
    'ACCESS-ESM1-5':  161,
    'IPSL-CM6A-LR' : 1910,
    'CanESM5'      : 5201,
    'MIROC-ES2L'   : 1850,
    'NorESM2-LM'   : 1600
}


startyear, endyear = refyear_dict[esm]+165, refyear_dict[esm]+165+84

# Load temperature
var = 'thetao'
fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                            diresgf=diresgf, version=version, 
                                            variant=variant, grid=grid)
new_fname_list = shorten_fname_list(fname_list, startyear, endyear)
zwds = xr.open_mfdataset(new_fname_list, **kwopenmfds)
zwds2 = zwds[var].to_dataset()
zwds2 = rename_vars_dims_coords(zwds2, rename_dict)
zwds2 = split_coords_dimensions(zwds2)
zwds_temp = shift_180_lon(zwds2)

# Load salinity
var = 'so'
fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                            diresgf=diresgf, version=version, 
                                            variant=variant, grid=grid)
new_fname_list = shorten_fname_list(fname_list, startyear, endyear)
zwds = xr.open_mfdataset(new_fname_list, **kwopenmfds)
zwds2 = zwds[var].to_dataset()
zwds2 = rename_vars_dims_coords(zwds2, rename_dict)
zwds2 = split_coords_dimensions(zwds2)
zwds_sali = shift_180_lon(zwds2)

# Load oxygen
var = 'o2'
fname_list = get_esgf_dataset_filepaths(var, esm, simu, 
                                            diresgf=diresgf, version=version, 
                                            variant=variant, grid=grid)
new_fname_list = shorten_fname_list(fname_list, startyear, endyear)
zwds = xr.open_mfdataset(new_fname_list, **kwopenmfds)
zwds2 = zwds[var].to_dataset()
zwds2 = rename_vars_dims_coords(zwds2, rename_dict)
zwds2 = split_coords_dimensions(zwds2)
zwds_oxyg = shift_180_lon(zwds2)

# save for later
zwds_attrs=zwds.attrs
year_list = ['%04d' %yyy for yyy in np.arange(startyear, endyear+.5)]

del zwds, zwds2
gc.collect()

# Loop on years
for year in year_list: 

    temp = zwds_temp.sel(time=year)['thetao'].load()
    sali = zwds_sali.sel(time=year)['so'].load()
    oxyg = zwds_oxyg.sel(time=year)['o2'].load()

    # Compute o2sat
    o2sat = get_o2sat_garcia(temp, sali, verbose=False).compute()

    # Clean
    del sali, temp
    gc.collect()

    # Compute aou
    aou = xr.zeros_like(o2sat)
    aou.values = o2sat.values - oxyg.values
    aou.attrs={}
    aou.attrs['units']='mol m-3'
    aou.attrs['long_name']='Apparent Oxygen Utilization'
    aou.attrs['description']='AOU computed as O2sat - O2 with O2sat computed following Garcia, H. E., & \
    Gordon, L. I. (1992). Oxygen solubility in seawater: Better fitting equations. Limnology and Oceanography, 37(6), 1307–1312.'
    aou = aou.rename('aou')

    # Clean
    del oxyg
    gc.collect()

    # Temporal mean
    o2sat_tavg = o2sat.groupby('time.year').mean(dim='time')
    aou_tavg = aou.groupby('time.year').mean(dim='time')

    # Clean
    del o2sat, aou
    gc.collect()

    # create dataset
    o2sat_ds = o2sat_tavg.to_dataset() 
    o2sat_ds.attrs = zwds_attrs
    aou_ds = aou_tavg.to_dataset() 
    aou_ds.attrs = zwds_attrs

    # Save in netcdf
    #---------------    
    # print('Save in netcdf...')
    ncname = netcdfdir+esm+'_'+simu+'_o2sat_'+year+'.nc'
    o2sat_ds.to_netcdf(ncname)
    # print('File saved: %s'%ncname)
    #---------------    
    # print('Save in netcdf...')
    ncname = netcdfdir+esm+'_'+simu+'_aou_'+year+'.nc'
    aou_ds.to_netcdf(ncname)
    # print('File saved: %s'%ncname)
#
print(f'Done')





2024-08-22 16:37:50.273700
# Compute AOU for piControl with specific years depending on models
## Compute
Done
peak memory: 5097.25 MiB, increment: 4539.54 MiB
CPU times: user 15min 16s, sys: 2min 27s, total: 17min 44s
Wall time: 18min 18s
